### Import libraries

In [1]:
%%capture libraries   

import sys
import os
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install mne
!{sys.executable} -m pip install mne-features
import numpy as np
import matplotlib 
import pathlib
import mne
import seaborn as sns
import pandas as pd
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, create_info, events_from_annotations
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs,corrmap
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch, tfr_stockwell,tfr_multitaper,tfr_array_morlet,AverageTFR
from scipy import signal, stats
from scipy.integrate import simps
#matplotlib.use('Qt5Agg') #allow interactive plots
import matplotlib.pyplot as plt
from mne.decoding import GeneralizingEstimator, Scaler,cross_val_multiscore, LinearModel, get_coef, Vectorizer, CSP, SlidingEstimator
from mne.viz import centers_to_edges
from mne.baseline import rescale
from sklearn import metrics, svm
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold, ShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_recall_fscore_support, precision_recall_curve, average_precision_score, plot_precision_recall_curve, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import plot_roc_curve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression 
import tensorflow as tf
import keras
from keras import layers as layers
from keras import optimizers
from keras.layers import Dense, Dropout, Flatten, LSTM, Activation, SpatialDropout1D,SpatialDropout2D, MaxPooling2D, MaxPooling1D
from keras.models import Sequential,Input,Model
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv1D, Conv2D
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from tensorflow.keras.utils import to_categorical
from keras.optimizers import SGD, RMSprop
%run SM1.ipynb import load_data, excl_chan, filter_data, make_epochs, plot_data, epochs_power
 

### Load epoched dataset

This dataset is filtered between 1-30Hz, artifacts correction with ICA and manual rejections of bad epochs (20 rejected). 

Each epoch has a duration of 9.5secs (9.998-0.25-0.25)

In [2]:
epochs= mne.read_epochs('epoched_data_280-epo.fif', preload=True)
print(epochs.get_data().shape)

Reading epoched_data_280-epo.fif ...
    Found the data of interest:
        t =     250.00 ...    9746.09 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
280 matching events found
No baseline correction applied
0 projection items activated
(280, 14, 2432)


In [3]:
#crop epoched data to reduce computational time

#epochs_cr = epochs.copy().crop(tmin=1., tmax=8.)
#print(epochs_cr.get_data().shape)

### Select occipital channels

In [4]:
epochs.drop_channels(['AF3','F7','F3','FC5','T7','P7','P8','T8','FC6','F4','F8','AF4']) 
print(epochs.get_data().shape)

(280, 2, 2432)


### Equalize the number of epochs count in each condition

In [5]:
mne.Epochs.equalize_event_counts(epochs, event_ids=epochs.event_id)
print(epochs.get_data().shape)

Dropped 4 epochs: 132, 167, 204, 271
(276, 2, 2432)


### Prepare dataset

In [6]:
#reshape dataset

epo=np.transpose(epochs, (0, 2, 1)) #epo, time, chans
x_data = np.array(epo) #input data
y_target = np.array(epochs.events[:,-1]) #target data
print(x_data.shape)
print(y_target.shape)
#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123) #training and testing


(276, 2432, 2)
(276,)


In [7]:
## Parse numbers as floats

x_data = np.asarray(x_data).astype('float32')
y_target = np.asarray(y_target).astype('float32')

In [8]:
#Standardise the input data

num_instances, num_time_steps, num_features = x_data.shape
x_data = np.reshape(x_data, (-1, num_features))
print(x_data.shape)
x_data = StandardScaler().fit_transform(x_data)
x_data = np.reshape(x_data, (num_instances, num_time_steps, num_features))
print(x_data.shape)

(671232, 2)
(276, 2432, 2)


In [11]:
n_timesteps, n_features, n_outputs = x_data.shape[1], x_data.shape[2], x_data.shape[2]
print(n_timesteps, n_features, n_outputs)

2432 2 2


### Define CNN architecture

In [19]:
def define_model():

    model=Sequential()
    model.add(Conv1D(filters=25, kernel_size=(11), strides=(1), padding='valid',  name='conv_1', input_shape=(n_timesteps, n_features)))

    model.add(LeakyReLU(alpha=0.015))
    model.add(SpatialDropout1D(rate=0.5, name='SpatialDropout_1'))

    model.add(Conv1D(filters=25, kernel_size=(1),  strides=(1), padding='valid', name='conv_2')) #spatial filter
    model.add(BatchNormalization(name='BatchNormalisation_1'))
    model.add(LeakyReLU(alpha=0.015))
    model.add(MaxPooling1D(pool_size=(3), strides=(1), padding='valid', name='MaxPooling_1'))

    model.add(Conv1D(filters=50, kernel_size=(11), strides=(1), padding='valid', name='conv_3')) #temporal filter
    model.add(LeakyReLU(alpha=0.015))
    model.add(SpatialDropout1D(rate=0.4, name='SpatialDropout_2'))
    model.add(MaxPooling1D(pool_size=(3), strides=(1), padding='valid', name='MaxPooling_2'))

    model.add(Conv1D(filters=100, kernel_size=(11), strides=(1), padding='valid', name='conv_4')) #temporal filter
    model.add(BatchNormalization(name='BatchNormalisation_2'))
    model.add(LeakyReLU(alpha=0.015))
    model.add(SpatialDropout1D(rate=0.5, name='SpatialDropout_3'))
    model.add(MaxPooling1D(pool_size=(3), strides=(1), padding='valid', name='MaxPooling_3'))

    model.add(Conv1D(filters=200, kernel_size=(11),  strides=(1), padding='valid', name='conv_5')) #temporal filter
    model.add(BatchNormalization(name='BatchNormalisation_3'))
    model.add(LeakyReLU(alpha=0.015))
   # model.add(SpatialDropout1D(rate=0.65, name='SpatialDropout_4'))
    model.add(MaxPooling1D(pool_size=(3), strides=(2), padding='valid', name='MaxPooling_4'))

    model.add(Flatten())

    model.add(Dense(1,activation="sigmoid")) 

    opt=optimizers.Adam(learning_rate=0.00003) 
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [24]:
model=define_model()
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv1D)              (None, 2422, 25)          575       
_________________________________________________________________
leaky_re_lu_60 (LeakyReLU)   (None, 2422, 25)          0         
_________________________________________________________________
SpatialDropout_1 (SpatialDro (None, 2422, 25)          0         
_________________________________________________________________
conv_2 (Conv1D)              (None, 2422, 25)          650       
_________________________________________________________________
BatchNormalisation_1 (BatchN (None, 2422, 25)          100       
_________________________________________________________________
leaky_re_lu_61 (LeakyReLU)   (None, 2422, 25)          0         
_________________________________________________________________
MaxPooling_1 (MaxPooling1D)  (None, 2420, 25)        

In [20]:
# Initialiase variables to store cross-validation results

acc_per_fold = []
loss_per_fold = []

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# Initialise the following parameters
no_epochs=20
batch_size= 32
inputs=x_data
targets=y_target
fold_no=1

In [21]:
for train, test in kfold.split(inputs, targets):
    model = define_model() #define model
    print('..................................................................')
    print(f'Training for fold {fold_no} ...')
    #fit data to model
    history = model.fit(inputs[train], targets[train], batch_size=batch_size, epochs=no_epochs, verbose=1)
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    #history_per_fold.append(history)
    
    # Increase fold number
    fold_no = fold_no + 1
  
    
# == Provide average scores ==
print('..................................................................')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('..................................................................')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('..................................................................')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('..................................................................')


..................................................................
Training for fold 1 ...
Epoch 1/20
7/7 [==============================] - 13s 2s/step - loss: 0.7613 - accuracy: 0.5119
Epoch 2/20
7/7 [==============================] - 10s 1s/step - loss: 0.9741 - accuracy: 0.4806
Epoch 3/20
7/7 [==============================] - 10s 1s/step - loss: 0.7812 - accuracy: 0.6031
Epoch 4/20
7/7 [==============================] - 11s 2s/step - loss: 0.7662 - accuracy: 0.5555
Epoch 5/20
7/7 [==============================] - 10s 1s/step - loss: 0.8061 - accuracy: 0.5214
Epoch 6/20
7/7 [==============================] - 10s 1s/step - loss: 0.6989 - accuracy: 0.5640
Epoch 7/20
7/7 [==============================] - 11s 2s/step - loss: 0.7386 - accuracy: 0.5341
Epoch 8/20
7/7 [==============================] - 10s 1s/step - loss: 0.7117 - accuracy: 0.5838
Epoch 9/20
7/7 [==============================] - 10s 1s/step - loss: 0.6519 - accuracy: 0.6588
Epoch 10/20
7/7 [============================